Le proponen automatizar el trabajo de un calderista. Sus variables son:

*   La **TEMPERATURA EXTERIOR** (exterior a la caldera, es decir, la temperatura de las habitaciones del edificio)
*   La **TEMPERATURA INTERIOR** (de la caldera)
*   El tamaño de la llama (**COMBUSTION**).



*   Cuando la **TEMPERATURA EXTERIOR** es **BAJA** , la **COMBUSTION** debe ser **ALTA**.
*   Cuando la **TEMPERATURA EXTERIOR** es **MEDIA**, la **COMBUSTION** debe ser **MODERADA**
*   Cuando la **TEMPERATURA EXTERIOR** es **ALTA**, la **COMBUSTION** debe estar en **PILOTO**.
*   Cuando la **TEMPERATURA INTERIOR** sea **ALTA**, la **COMBUSTION** debe **LIMITARSE LIGERAMENTE**
*   Cuando la **TEMPERATURA INTERIOR** sea **CRITICA**, la **COMBUSTION** debe ser **REDUCIDA**
*   Si la **TEMPERATURA INTERIOR** no es **ALTA o CRITICA**, **no debería interesarnos**

Explicite las reglas que ha elegido para esta tarea, y de ejemplos de valores de entrada y salida.

In [ ]:
!pip install scikit-fuzzy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz

In [ ]:
#Rango de variables de entrada
t_ext = np.arrange(-20, 50, 1)
t_int = np.arrange(30, 100, 1)

#Rango de variable de salida
combustion = np.arrange(-5,10)

#Funcion de pertenencia de temperatura exterior
t_ext_baja = fuzz.gaussmf(t_ext,[-20,10])
t_ext_media = fuzz.gaussmf(t_ext,[11,22])
t_ext_alta = fuzz.gaussmf(t_ext,[23,50])

#Funcion de pertenencia de temperatura interior
t_int_baja = fuzz.gaussmf(t_int,[30,50])
t_int_alta = fuzz.gaussmf(t_int,[51,70])
t_int_critica = fuzz.gaussmf(t_int,[71,100])


